In [1]:
# Encrypted Messaging App Prototype (RSA + AES)

from cryptography.hazmat.primitives.asymmetric import rsa, padding as rsa_padding
from cryptography.hazmat.primitives.asymmetric import padding as asym_padding
from cryptography.hazmat.primitives import serialization, hashes
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding as sym_padding
from cryptography.hazmat.backends import default_backend
from os import urandom
import os

# Step 1: RSA key pair generated by User A  
def generate_rsa_keys():
    private_key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )
    public_key = private_key.public_key()
    return private_key, public_key

# Saving of RSA keys to PEM files
def save_keys(private_key, public_key):
    with open("user_a_private_key.pem", "wb") as f:
        f.write(private_key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        ))
    with open("user_a_public_key.pem", "wb") as f:
        f.write(public_key.public_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PublicFormat.SubjectPublicKeyInfo
        ))

# Step 2: User B use AES-256 to encrypt message 
def encrypt_message(public_key):
    if not os.path.exists("message.txt"):
        with open("message.txt", "w") as f:
            f.write("The most secret 2nd Text in Univers. Nobady should read it!")

    with open("message.txt", "rb") as f:
        plaintext = f.read()

    aes_key = urandom(32)  # AES-256
    iv = urandom(16)

    padder = sym_padding.PKCS7(128).padder()
    padded_plaintext = padder.update(plaintext) + padder.finalize()

    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_plaintext) + encryptor.finalize()

    with open("encrypted_message.bin", "wb") as f:
        f.write(iv + ciphertext)  # Save IV with ciphertext

    encrypted_aes_key = public_key.encrypt(
        aes_key,
        asym_padding.OAEP(
            mgf=asym_padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    with open("aes_key_encrypted.bin", "wb") as f:
        f.write(encrypted_aes_key)

# Step 3: User A use private key to decrypt AES key and then message 
def decrypt_message(private_key):
    with open("aes_key_encrypted.bin", "rb") as f:
        encrypted_aes_key = f.read()

    aes_key = private_key.decrypt(
        encrypted_aes_key,
        asym_padding.OAEP(
            mgf=asym_padding.MGF1(algorithm=hashes.SHA256()),
            algorithm=hashes.SHA256(),
            label=None
        )
    )

    with open("encrypted_message.bin", "rb") as f:
        iv_ciphertext = f.read()
    iv = iv_ciphertext[:16]
    ciphertext = iv_ciphertext[16:]

    cipher = Cipher(algorithms.AES(aes_key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()

    unpadder = sym_padding.PKCS7(128).unpadder()
    plaintext = unpadder.update(padded_plaintext) + unpadder.finalize()

    with open("decrypted_message.txt", "wb") as f:
        f.write(plaintext)

# Full process
if __name__ == "__main__":
    # generation of keys by User A
    priv_key, pub_key = generate_rsa_keys()
    save_keys(priv_key, pub_key)

    # encryption by User B 
    encrypt_message(pub_key)

    # decryption of  message by User A
    decrypt_message(priv_key)

    print("Process of Encryption / decryption is complete")


Process of Encryption / decryption is complete


Explanation of Encryption Flow 
Mini encrypted messaging system allows exchange of secure messages using RSA and AES hybrid encryption by combining asymmetric and symmetric cryptography.  AES-256 is used for secured and fast encryption, and RSA - for secure transmitting of AES key, providing key exchange security and confidentiality.   
According task, User A generates 2048-bit RSA key pair. private key is saved as user_a_private_key.pem and public key is saved as user_a_public_key.pem. Then shares public key to User B. 
User B generates random 256-bit AES key and encrypts message.txt file using AES-256 in CBC mode with padding PKCS#7. As result gets encrypted_message.bin. Then encrypts AES key using RSA (User A's public key) and gets aes_key_encrypted.bin. 
User B securely sends both encrypted files to User A, who receives and decrypts AES key using own private RSA key. Then AES key is used to decrypt encrypted_message.bin and final plaintext is saved as decrypted_message.txt.   
Output Files
-	user_a_public_key.pem  
-	user_a_private_key.pem  
-	message.txt  
-	encrypted_message.bin  
-	aes_key_encrypted.bin  
-	decrypted_message.txt  